In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Kaggle Challenge
#Suppress the warning messages
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
train_data=pd.read_csv('/kaggle/input/mlb-player-digital-engagement-forecasting/train.csv',encoding='latin')
test_data=pd.read_csv('/kaggle/input/mlb-player-digital-engagement-forecasting/example_test.csv',encoding='latin')

In [ ]:
train_data.head()

In [ ]:
#Converting the json file to a dataframe
import json
from pandas.io.json import json_normalize
json_string=train_data['nextDayPlayerEngagement']
print(json_string)

In [ ]:
def json_to_dataframe (dataframe, column):
    num_rows = len(dataframe)
    
    data_list = []
    for row in tqdm(range(num_rows)):
        json_data = dataframe.iloc[row][column]
        if str(json_data) != "NaN":
            data = pd.read_json(json_data)
            data_list.append(data)
        
    all_data = pd.concat(data_list, axis = 0)
    
    return all_data

In [ ]:
player_engagement_data=json_to_dataframe(train_data,'nextDayPlayerEngagement')
player_engagement_data=pd.DataFrame(player_engagement_data)

In [ ]:
player_engagement_data.head()

In [ ]:
df=pd.DataFrame(player_engagement_data)
df['engagementMetricsDate'] = pd.to_datetime(df['engagementMetricsDate'])
df['new_date'] = df['engagementMetricsDate'].astype('int64')
df.info()

In [ ]:
df.head()

In [ ]:
df['new_date']=df['new_date']/100000000000

In [ ]:
df.head()

In [ ]:
df['date_playerId']=df.new_date.astype(str)+'_'+df.playerId.astype(str)
new_dataframe=df

In [ ]:
new_dataframe.head()

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
#Random Forest Regressor for target1, target2, target3, target4
TargetVariable=['target1','target2','target3','target4']
Predictors=['new_date','playerId']
X=new_dataframe[Predictors].values
y=new_dataframe[TargetVariable].values

# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
#Scaling the data
from sklearn.preprocessing import StandardScaler, MinMaxScaler
PredictorScaler=StandardScaler()
PredictorScalerFit=PredictorScaler.fit(X)
X=PredictorScalerFit.transform(X)

from sklearn.ensemble import RandomForestRegressor
clf =MultiOutputRegressor( RandomForestRegressor(max_depth=6,n_estimators=200,criterion='mse',oob_score=True))
print(clf)
RF=clf.fit(X_train,y_train)
prediction=RF.predict(X_test)

In [ ]:
def my_custom_loss_func(y_test, prediction):
    diff = np.abs(y_test - prediction).max()
    return np.log1p(diff)

In [ ]:
my_custom_loss_func(y_test,prediction)

In [ ]:
from sklearn.metrics import fbeta_score, make_scorer
score = make_scorer(my_custom_loss_func, greater_is_better=False)
score(clf,X_train,y_train)

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test,prediction))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, prediction))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, prediction)))

In [ ]:
prediction=pd.DataFrame(prediction)
prediction.head()

In [ ]:
prediction.columns=['targetpred1','targetpred2','targetpred3','targetpred4']
prediction.head()

In [ ]:
X_test=pd.DataFrame(X_test)
X_test.head()

In [ ]:
X_test.columns=['new_date','playerId']
X_test['date_playerId']=X_test.new_date.astype(str)+'_'+X_test.playerId.astype(str)
X_test.head()

In [ ]:
X_train=pd.DataFrame(X_train)
X_train.head()
#X_train.columns=['new_date','playerId']
#X_train['date_playerId']=X_train.new_date.astype(str)+'_'+X_train.playerId.astype(str)
#X_train.head()

In [ ]:
output_=X_test[['new_date','date_playerId']]
output_.head()

In [ ]:
output_['target1_pred']=prediction.targetpred1.astype(str)
output_['target2_pred']=prediction.targetpred2.astype(str)
output_['target3_pred']=prediction.targetpred3.astype(str)
output_['target4_pred']=prediction.targetpred4.astype(str)
output_.head()

In [ ]:
output_.to_csv(r'submission.csv')